# **Sleep Detection**

Göz kırpma sayıları anlaşılmaya çalışılacaktır.

Göz kırpma sayıları ve gözün kapalı kalma süresi değerlendirilerek uyku tespiti yapılacaktır.

In [ ]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
from cvzone.PlotModule import LivePlot

In [ ]:
cap = cv2.VideoCapture(r"C:\Users\ilhan\Desktop\video1.mp4")
detector = FaceMeshDetector()
plotY = LivePlot(540, 360, [10, 60])

In [ ]:
#id list ile face mesh işlemi yapılırken göze odaklanılır.
#bu yüzden gözdeki noktaların idleri alınmıştır.
#sol göz ile işlem yapılmıştır.
id_list = [22, 23, 24, 26, 110, 157, 158, 159, 160, 161, 130, 243]
color = (0,0,255)
ratio_list = []
counter = 0
blink_counter = 0

In [ ]:
while True:
    ret, frame = cap.read()
    
    #yüzdeki mesh pointler tespit edilir.
    frame, faces = detector.findFaceMesh(frame, draw = False)
    
    if ret == True:
    
        if faces:
            face = faces[0]

            for id in id_list:
                cv2.circle(frame, face[id], 5, color, cv2.FILLED)
            #sol gözün etrafındaki mesh pointler tespit edilmiştir.
            
            #gözün kapalı olup olmadığının tespit edilmesi için uç noktalarının idleri alınır.
            left_up = face[159]
            left_down = face[23]
            left_left = face[130]
            left_right = face[243]
            
            #uç noktaların yatay ve dikey olarak farkları alınır.
            length_vertical, _ = detector.findDistance(left_up, left_down)
            length_horizontal, _ = detector.findDistance(left_left, left_right)
            
            #uç noktalar arasına yatay ve dikey olmak üzere 2 tane yeşil bir line çizilir.
            cv2.line(frame, left_up, left_down, (0,255,0),3)
            cv2.line(frame, left_left, left_right, (0,255,0),3)
            
            #yatay ve dikey uzaklıklar arasındaki orana bakılarak gözün açık ya da kapalı olduğu anlaşılacaktır.
            ratio = int((length_vertical/length_horizontal)*100)
            ratio_list.append(ratio)
            #print(ratio_list)
            
            #ratio_list'in uzunluğu 3'ten fazla 0. indeksi listeden atar.
            #bu durumda ratio_list tüm değerlerin ortalaması değil de 3 değerin ortalamasına göre hesaplanır.
            #her adımda bir sonraki indeks ile devam eder.
            #eğer bu koşul belirlenmezse liste çok dolar ve hiçbir zaman thresholdun altına düşmez.
            if len(ratio_list)>3:
                ratio_list.pop(0)       
            
            #oranın ortalaması alınır.
            ratio_avg = sum(ratio_list)/len(ratio_list)
            #print(ratio_avg)
            
            #ortalamaya göre bir threshold belirlenir ve bu threshold'dan küçük olduğunda göz kapalı durumdadır.
            #göz kapandığında counter artırılır.
            if ratio_avg < 35 and counter == 0:
                blink_counter += 1
                color = (0,255,0)
                counter = 1
            if counter != 0:
                counter += 1
                if counter > 10:
                    counter = 0
                    color = (0,0,255)

            cvzone.putTextRect(frame, f'Blink Count: {blink_counter}', (50,100), colorR = color)

            img_plot = plotY.update(ratio_avg, color)
            frame = cv2.resize(frame, (640,360))
            img_stack = cvzone.stackImages([frame, img_plot], 2,1)

        cv2.imshow("Sleep Detection", img_stack)
        
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break
            
    else:break
    
cap.release()
cv2.destroyAllWindows()